### Face Similarity Caculation for crawled images
- Loading the extracted name from txt file
- If there are no paired images in the training dataset, we crawled image using google_crawler. The extracted person's name will be used as the keyword for image crawling.
- Extract crawled face images paired with the extracted name
- Caculate the face similarity between the face images in the test dataset and images paired with the extracted name in the crawled dataset. 

In [ ]:
import os
def extract_text (orig_file):
    articles_names=open(orig_file, 'r', encoding="utf-8")
    next(articles_names)
    lines = [line.strip() for line in articles_names]
    result=[]
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        result.append((segs[0], segs[3].split(",")[0]))
    return result

In [ ]:
ar_name_list=extract_text("test_title_eng_name.tsv")

In [ ]:
len(ar_name_list)

In [ ]:
d='faces'
sub_directories = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
idx_name={}
name_idx={}
idx=1
for sub_dir in sub_directories:
    idx_name[sub_dir]='face_'+str(idx)
    name_idx['face_'+str(idx)] = sub_dir
    idx+=1
name_idx

In [ ]:
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]

In [ ]:
import icrawler
from icrawler.builtin  import GoogleImageCrawler

google_crawler = GoogleImageCrawler(feeder_threads=1,parser_threads=2,downloader_threads=4,storage={'root_dir': 'name_no_img'})
filters = dict(date=((2019, 1, 1), (2020, 11, 30)))
google_crawler.crawl(keyword='sunny', filters=filters, max_num=10, file_idx_offset=0)

In [ ]:
import os
from deepface import DeepFace
cnt = 0
img_files = [f for f in os.listdir("test_img_face") if f.endswith('.jpg')]
ar_img_files={}
f = open("name_no_img.txt", "a")
for ar_name in ar_name_list:
    if ar_name[1].strip() != 'NA' and ar_name[1] in idx_name:
        if os.path.exists(os.path.join("mapped_face", idx_name[ar_name[1]])):
            print("found")
    elif ar_name[1].strip() != 'NA':
        if not os.path.isdir(os.path.join('name_no_img', ar_name[1])):
            os.mkdir(os.path.join('name_no_img', ar_name[1]))
        google_crawler = GoogleImageCrawler(feeder_threads=1,parser_threads=2,downloader_threads=4,storage={'root_dir': os.path.join('name_no_img', ar_name[1])})
        filters = dict(date=((2019, 1, 1), (2021, 7, 30)))
        google_crawler.crawl(keyword=ar_name[1], filters=filters, max_num=5, file_idx_offset=0)
f.close()

In [ ]:
import cv2
import time
import matplotlib.pyplot as plt
def detect_face_cv(file):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # Read the input image
    img = cv2.imread(file)
    width, height, channels=img.shape
    # Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces= face_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw rectangle around the faces
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        if w!=width or height!=h:
            return True
    return False

In [ ]:
from deepface import DeepFace
def deep_detect_backend(file):
    backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']
    c=0
    for backend in backends:
        try:
            detected_face = DeepFace.detectFace(file, detector_backend=backend)
        except:
            c+=1
    if c== len(backends):
        return False
    else:
        return True

In [ ]:
from deepface import DeepFace
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
def deep_detect(file):
    c=0
    for model in models:
        try:
            detected_face = DeepFace.detectFace(file, model_name=model)
        except:
            c+=1
    if c== len(models):
        return False
    else:
        return True

In [ ]:
d='name_no_img'
sub_directories = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
idx_name={}
name_idx={}
idx=1
for sub_dir in sub_directories:
    idx_name[sub_dir]='crawl_face_'+str(idx)
    name_idx['crawl_face_'+str(idx)] = sub_dir
    idx+=1
name_idx

In [ ]:
idx_name

In [ ]:
import os
import shutil
def copy_all_files(src, dest, idx_name):
    src_files = os.listdir(src)
    for file_name in src_files:
        full_file_name = os.path.join(src, file_name)
        dest_folder=os.path.join(dest, idx_name[os.path.basename(src)])
        if not os.path.isdir(dest_folder):
            os.mkdir(dest_folder)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dest_folder)

In [ ]:
d='name_no_img'
sub_full_paths = [os.path.join(d, o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
sub_full_paths
mapped_folder='crawl_mapped_face'
for sub_dir in sub_full_paths:
    copy_all_files(sub_dir, mapped_folder, idx_name)

In [ ]:
import shutil
path="crawl_mapped_face"
dest_folder="name_no_img_fp"
sub_directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
c=0
for sub_dir in sub_directories:
        files = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith('.jpg')]
        for file in files:
            if os.path.isdir(sub_dir) and not deep_detect_backend(file) and not deep_detect(file) and not detect_face_cv(file):
                print(sub_dir)
                shutil.rmtree(sub_dir) 
#f.close()

In [ ]:
df = DeepFace.find(img_path = r"test_img_face\139017.jpg", db_path =r"crawl_mapped_face\crawl_face_4", model_name='Facenet', enforce_detection=False)

In [ ]:
print(df['Facenet_cosine'].mean())

In [ ]:
len(df)

In [ ]:
import os
from deepface import DeepFace
cnt = 0
img_files = [f for f in os.listdir("test_img_face") if f.endswith('.jpg')]
ar_img_files={}
for ar_name in ar_name_list:
    if ar_name[1].strip() != 'NA' and ar_name[1] in idx_name:
        if os.path.exists(os.path.join("crawl_mapped_face", idx_name[ar_name[1]])):
            df_results=[]
            for img_file in img_files:
                img_path=os.path.join("test_img_face", img_file)
                df = DeepFace.find(img_path = img_path, db_path =os.path.join("crawl_mapped_face", idx_name[ar_name[1]]), model_name='Facenet', enforce_detection=False)
                if len(df)>0:
                    df_results.append((img_path,df['Facenet_cosine'].mean(), len(df)))
                else:
                    df_results.append((img_path,"NA", 0))
            ar_img_files[ar_name[0]]=df_results
            cnt+=1
            print(cnt, " name completed")

In [ ]:
ar_img_files

In [ ]:
f = open("crawl_img_similarity.txt", "a")
for key, v in ar_img_files.items():
    for item in v:
        result=key+"\t"+os.path.basename(item[0])+"\t"+str(item[1])+"\t"+str(item[2])+"\n"
        f.write(result)
f.close()

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [ ]:
import os
def get_caption (orig_file):
    articles_names=open(orig_file, 'r', encoding="utf-8")
    lines = [line.strip() for line in articles_names]
    result_dict={}
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        if segs[0] not in result_dict:
            result_dict[segs[0]]=segs[1]
    return result_dict

In [ ]:
caption_dict=get_caption ("test_img\image_caption_result.txt")

In [ ]:
import os
def get_id_title (orig_file):
    articles_names=open(orig_file, 'r', encoding="utf-8")
    next(articles_names)
    lines = [line.strip() for line in articles_names]
    result_dict={}
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        if segs[0] not in result_dict:
            result_dict[segs[0]]=segs[2]
    return result_dict

In [ ]:
id_title=get_id_title ("test_title_eng.tsv")

In [ ]:
id_title

In [ ]:
sim_result={}
cnt=0
for ar_id, title in id_title.items():
    caption_sim=[]
    for img_id, caption in caption_dict.items():
        sim = cosine_sim(title, caption)
        caption_sim.append((img_id, sim))
    cnt+=1
    print(cnt," article: ",ar_id)
    sim_result[ar_id]=caption_sim

In [ ]:
print(len(sim_result))

In [ ]:
f = open("caption_similarity.txt", "a")
for key, v in sim_result.items():
    for item in v:
        result=key+"\t"+os.path.basename(item[0])+"\t"+str(item[1])+"\n"
        f.write(result)
f.close()

In [ ]:
sim_result_gen={}
cnt=0
for ar_id, title in id_title.items():
    caption_sim=[]
    for img_id, caption in caption_dict.items():
        sim = cosine_sim(title, caption)
        caption_sim.append((img_id, sim))
    cnt+=1
    print(cnt," article: ",ar_id)
    sim_result_gen[ar_id]=caption_sim

In [ ]:
cnt=0
for k, v in sim_result.items():
    count = sum(value[1] == 0 for value in sim_result[k])
    if count == 1913:
        cnt+=1
print(cnt)